<a href="https://colab.research.google.com/github/Get-the-CODE/Machine-Learning-codes/blob/main/Weights_%26_Biases_26June.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.1/300.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00


In [3]:
import wandb


In [4]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.cluster import KMeans
from sklearn import datasets, cluster

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [6]:
from google.colab import files

uploaded = files.upload()

Saving train.csv to train.csv


In [8]:
from google.colab import files

uploaded = files.upload()

Saving test.csv to test.csv


In [7]:
import pandas as pd

# Read the CSV file into a DataFrame
df_train = pd.read_csv('train.csv')

In [9]:
import pandas as pd

# Read the CSV file into a DataFrame
df_test = pd.read_csv('test.csv')

In [10]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
df_train['Salutation'] = df_train['Name'].apply(lambda name: name.split(',')[1].split('.')[0].strip())
df_train['Salutation'] = df_train['Salutation'].replace('Mme','Mrs')
df_train['Salutation'] = df_train['Salutation'].replace(['Mlle','Ms'], 'Miss')
lst = ['Mr','Miss','Mrs','Master']
df_train['Salutation'] = df_train['Salutation'].map(lambda x: x if x in lst else 'others')
age_title_train = df_train.groupby('Salutation')['Age'].median()
df_train['Age'] = np.where(df_train['Age'].isnull(),age_title_train[df_train['Salutation']],df_train['Age'])
fare_class_train = df_train.groupby('Pclass')['Fare'].mean()
df_train['Fare'] = np.where(df_train['Fare'].isnull(),fare_class_train[df_train['Pclass']],df_train['Fare'])
df_train = df_train.dropna(subset=['Embarked'])
df_train.drop('Cabin',axis=1, inplace= True)
df_train['Family'] = df_train['SibSp'] + df_train['Parch']
df_train['Family'] = df_train['Family'].map(lambda x: "Alone" if x==0 else "With Family")
df_train['Pclass'] = df_train['Pclass'].map(lambda x: "First_Class" if x == 1 else ("Second_Class" if x == 2 else "Third_Class"))


# Function to calculate WoE for a single feature
def calculate_woe(df, feature, target):
    total_events = df[target].sum()
    total_non_events = df[target].count() - total_events

    # Group by feature and calculate event and non-event counts
    df_woe = df.groupby(feature).agg({
        target: ['sum', 'count']
    })
    df_woe.columns = ['events', 'total']
    df_woe['non_events'] = df_woe['total'] - df_woe['events']

    # Calculate event rate, non-event rate, and WoE
    df_woe['event_rate'] = df_woe['events'] / total_events
    df_woe['non_event_rate'] = df_woe['non_events'] / total_non_events
    df_woe['woe'] = np.log((df_woe['event_rate'] + 1e-10) / (df_woe['non_event_rate'] + 1e-10))

    return df_woe['woe']

# Function to apply WoE transformation to the categorical DataFrame
def apply_woe(df, features, target):
    df_woe = df.copy()
    for feature in features:
        woe_values = calculate_woe(df, feature, target)
        df_woe[feature] = df[feature].map(woe_values)
    return df_woe

# List of categorical features
features = ['Pclass', 'Sex', 'Embarked', 'Salutation','Family']

# Apply WoE transformation to the combined DataFrame
df_woe = apply_woe(df_train, features, 'Survived')

df_woe.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)

X = df_woe.drop(columns=['Survived'])
y = df_woe['Survived']


df_woe2 = df_woe.drop(['SibSp', 'Parch', 'Sex', 'Fare'], axis=1)
X2 = df_woe2.drop(columns=['Survived'])
y2 = df_woe2['Survived']


import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# Define features and target

df_woe3 = df_woe.drop(['SibSp', 'Parch','Sex','Fare'], axis=1)
X = df_woe3.drop(columns=['Survived'])
y = df_woe3['Survived']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Set the desired maximum depth for the gradient boosting classifier
max_tree_depth = 4  # Change this value to set a different max depth

# Define the pipeline
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', GradientBoostingClassifier(random_state=42, max_depth=max_tree_depth))
])

# Fit the pipeline
pipeline.fit(X_train, y_train)

#------------------------------------

import joblib
from sklearn import metrics

# Save the model as a pickle in a file
joblib.dump(pipeline, 'GBMclassifier.pkl')

# Output confirmation message
print("Model architecture saved successfully.")


#---------------------------------------------------------

<ipython-input-11-528f761f6045>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop('Cabin',axis=1, inplace= True)
<ipython-input-11-528f761f6045>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Family'] = df_train['SibSp'] + df_train['Parch']
<ipython-input-11-528f761f6045>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

Model architecture saved successfully.


In [12]:
df_test['Salutation'] = df_test['Name'].apply(lambda name: name.split(',')[1].split('.')[0].strip())
df_test['Salutation'] = df_test['Salutation'].replace('Mme','Mrs')
df_test['Salutation'] = df_test['Salutation'].replace(['Mlle','Ms'], 'Miss')
lst = ['Mr','Miss','Mrs','Master']
df_test['Salutation'] = df_test['Salutation'].map(lambda x: x if x in lst else 'others')
age_title_test = df_test.groupby('Salutation')['Age'].median()
df_test['Age'] = np.where(df_test['Age'].isnull(),age_title_test[df_test['Salutation']],df_test['Age'])
fare_class_test = df_test.groupby('Pclass')['Fare'].mean()
df_test['Fare'] = np.where(df_test['Fare'].isnull(),fare_class_test[df_test['Pclass']],df_test['Fare'])
df_test = df_test.dropna(subset=['Embarked'])
df_test.drop('Cabin',axis=1, inplace= True)
df_test['Family'] = df_test['SibSp'] + df_test['Parch']
df_test['Family'] = df_test['Family'].map(lambda x: "Alone" if x==0 else "With Family")
df_test['Pclass'] = df_test['Pclass'].map(lambda x: "First_Class" if x == 1 else ("Second_Class" if x == 2 else "Third_Class"))


# Function to calculate WoE for a single feature
def calculate_woe(df_test, feature, target):
    total_events = df_test[target].sum()
    total_non_events = df_test[target].count() - total_events

    # Group by feature and calculate event and non-event counts
    df_woe_test = df_test.groupby(feature).agg({
        target: ['sum', 'count']
    })
    df_woe_test.columns = ['events', 'total']
    df_woe_test['non_events'] = df_woe_test['total'] - df_woe_test['events']

    # Calculate event rate, non-event rate, and WoE
    df_woe_test['event_rate'] = df_woe_test['events'] / total_events
    df_woe_test['non_event_rate'] = df_woe_test['non_events'] / total_non_events
    df_woe_test['woe'] = np.log((df_woe_test['event_rate'] + 1e-10) / (df_woe_test['non_event_rate'] + 1e-10))

    return df_woe_test['woe']

# Function to apply WoE transformation to the categorical DataFrame
def apply_woe(df_test, features, target):
    df_woe_tst = df_test.copy()
    for feature in features:
        woe_values = calculate_woe(df_test, feature, target)
        df_woe_tst[feature] = df_test[feature].map(woe_values)
    return df_woe_tst

# List of categorical features
features = ['Pclass', 'Sex', 'Embarked', 'Salutation','Family']

# Apply WoE transformation to the combined DataFrame
df_woe_test = apply_woe(df_test, features, 'Survived')

df_woe_test.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)

X_test = df_woe_test.drop(columns=['Survived'])
y_test = df_woe_test['Survived']


df_woe_test2 = df_woe_test.drop(['SibSp', 'Parch', 'Sex', 'Fare'], axis=1)
X2_test = df_woe_test2.drop(columns=['Survived'])
y2_test = df_woe_test2['Survived']


#--------------------------------------------------------------------


import joblib
from sklearn import metrics

GBMreg = joblib.load('GBMclassifier.pkl')

# Make predictions
y_pred = GBMreg.predict(X2_test)

# Evaluate the model using Cohen's kappa score
kappa_score = metrics.cohen_kappa_score(y2_test, y_pred)
print("Cohen's kappa score:", kappa_score)

#-------------------------------------------------------------------



Cohen's kappa score: 0.8153998025666338
